### Import Modules

In [1]:
import numpy as np
import pandas as pd

import folium
from folium.plugins import MarkerCluster
from folium.features import ColorLine

In [2]:
fp = '../data/route45_xyz.csv'
route_45_full = pd.read_csv(fp)
route_45_full.head()

,FID,Id,ORIG_FID,Id_1,Z (ft),POINT_X,POINT_Y,Lat,Long
0,0,0,0,0,221.743114,1277982.100,240941.2000,47.651001,-122.304001
1,1,0,0,0,89.887224,1277970.258,240910.6033,47.651001,-122.304001
2,2,0,0,0,222.650173,1277958.417,240880.0065,47.650902,-122.304001
3,3,0,0,0,42.084594,1277946.575,240849.4098,47.650799,-122.304001
4,4,0,0,0,222.277597,1277934.733,240818.8131,47.650700,-122.304001


In [3]:
# Extract xyz data from the full route data
route_45 = route_45_full[['Lat', 'Long', 'Z (ft)']]
route_45 = route_45.rename(columns = {'Z (ft)':'Alt'})
route_45.head()

,Lat,Long,Alt
0,47.651001,-122.304001,221.743114
1,47.651001,-122.304001,89.887224
2,47.650902,-122.304001,222.650173
3,47.650799,-122.304001,42.084594
4,47.650700,-122.304001,222.277597


In [4]:
# Determine Map center
route_45_center = route_45.mean()
route_45_center_coord = [route_45_center.values[0], route_45_center.values[1]]

#Create the map
route_45_map = folium.Map(location = route_45_center_coord, zoom_start = 12)

#Display the map
# route_45_map

In [57]:
route_45_xyz = np.array(route_45)
route_45_xy = route_45_xyz[:, :2].tolist()
route_45_z = route_45_xyz[:,2].tolist()
route_line = folium.PolyLine(route_45_xy, weight=3, color='red').add_to(route_45_map)
# #Display the map
route_45_map
# route_45_xy[:5]

In [58]:
def alt_color(route_45_z):
    colors = ['#0000FF', '#0040FF', '#0080FF', '#00FFB0', '#00E000',
             '#80FF00', '#FFFF00', '#FFC000', '#FF0000']   
    route_alt_range = np.linspace(min(route_45_z), max(route_45_z), len(colors))         
    color_alt = []
    for idx in range(len(route_45_z) - 1):
        alt_1 = route_45_z[idx]
        index_1 = np.where(route_alt_range <= alt_1)[0][-1]
        color_alt.append(colors[index_1])
        
    return color_alt

In [59]:
colors_alt = alt_color(route_45_z)
route_alt = ColorLine(route_45_xy, colors=list(range(0, len(route_45_z))),
              colormap=colors_alt, weight=3).add_to(route_45_map)

route_45_map.fit_bounds(route_45_map.get_bounds())
route_45_map  

In [ ]:
# fp = '../data/Transit_Routes_for_King_County_Metro__transitroute_line.shp'
# data = gpd.read_file(fp)
# r45 = data[data['ROUTE_NUM']==45]
# gjson = r45.to_json()
# points = folium.GeoJson(gjson)
# m.add_child(points)